<a href="https://colab.research.google.com/github/jt-schmidt/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-14 15:24:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  3.96MB/s    in 0.2s    

2020-11-14 15:24:06 (3.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#Deliverable 1.6
#First extract one of the review datasets, then create a new DataFrame.
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
from pyspark.sql.functions import to_date
import pandas as pd
# Read in the Review dataset as a DataFrame
amazon_review_df = df
amazon_review_df.show()
#Dataframe type check:  pandas vs spark
if isinstance(amazon_review_df, pd.DataFrame):
    print('pandas')
else:
    print('spark')

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [34]:
#Deliverable 1
# Create the vine_table. DataFrame
# vine_df = df.select([])

#SQL from pgAdmin BigDataChallenge DB
#CREATE TABLE vine_table (
#  review_id TEXT PRIMARY KEY,
#  star_rating INTEGER,
#  helpful_votes INTEGER,
#  total_votes INTEGER,
#  vine TEXT,
#  verified_purchase TEXT
from pyspark.sql.functions import desc
from pyspark.sql.functions import to_date
import pyspark.sql.functions as F

vine_df = amazon_review_df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2E5BQMWTW9JUQ|          5|            0|          0|   N|                Y|
|R3VTIZI7XPW9PR|          4|            0|          0|   N|                Y|
|R1UC7WJRQQKU5V|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3V0YK55IDIW3T|          5|            0|          0|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R14Z48EC6DKJX9|          5|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R3IU97WRKS5SLN|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|     

In [35]:
#Deliverable 2.1
#Filter data & create new dataframe where total_votes >= 20
#df.filter((df.d<5)& (df.col2!=df.col4) & (df.col1==df.col3)).show()

vine_vote_over20_df = vine_df.filter((vine_df.total_votes>=20))
vine_vote_over20_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EI0JMVNT5XO3|          5|           20|         20|   N|                Y|
|R2PDRZJOVO772P|          1|            2|         20|   N|                Y|
|R15W8DZ57HXZ28|          5|           19|         20|   N|                Y|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R1Y6ZYTARUH13D|          5|           14|         20|   N|                N|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
| RQ4NYV9QVM2IJ|          1|            0|         20|   N|                N|
|R1RA7M526HB0O7|          1|            7|         20|   N|                N|
| R7UQ7ME8NFJMX|          1|            0|         20|   N|                N|
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|     

In [38]:
#Deliverable 2.2
#Filter vine_vote_over20 where # helpful_votes / total_votes >= 50%

vine_helpful_over50percent_df = vine_vote_over20_df.filter( ( vine_vote_over20_df.helpful_votes / vine_vote_over20_df.total_votes ) >= 0.5 )
vine_helpful_over50percent_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2G8UKUJZEL4EB|          5|           19|         20|   N|                Y|
| RNE0A8LOLHTNG|          2|           14|         20|   N|                N|
|R3EI0JMVNT5XO3|          5|           20|         20|   N|                Y|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R15W8DZ57HXZ28|          5|           19|         20|   N|                Y|
|R18UIVCYVG6HKQ|          3|           13|         20|   N|                N|
|R1Y6ZYTARUH13D|          5|           14|         20|   N|                N|
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R26OQY0D8FB0TN|          1|           16|         20|   N|                Y|
|R1N1YR7XNB54BF|          5|           20|         20|   N|     

In [41]:
#Deliverable 2.3
#Filter vine_helpful_over50percent_df where review was written as part of Vine program (vine == 'Y')

vine_Y_df = vine_helpful_over50percent_df.filter(  vine_helpful_over50percent_df.vine == 'Y' )
vine_Y_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R0F8ETAGTIOT|          5|           14|         20|   Y|                N|
|R2INH817LTJFGI|          5|           18|         20|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|                N|
|R3N4QAXTQCA1ED|          5|           18|         20|   Y|                N|
|R369DSVA6AH6SB|          5|           16|         21|   Y|                N|
|R3QJ1YLOZNKQ4L|          5|           19|         22|   Y|                N|
| RF41VQVEOQ4DV|          5|           17|         22|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| REAX2MY43TDPR|          2|           19|         23|   Y|                N|
|R2JJLTBVU6R54W|          5|           16|         23|   Y|     

In [42]:
#Deliverable 2.4
#Filter vine_helpful_over50percent_df where review was written was NOT part of Vine program (vine == 'N')

vine_N_df = vine_helpful_over50percent_df.filter(  vine_helpful_over50percent_df.vine == 'N' )
vine_N_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2G8UKUJZEL4EB|          5|           19|         20|   N|                Y|
| RNE0A8LOLHTNG|          2|           14|         20|   N|                N|
|R3EI0JMVNT5XO3|          5|           20|         20|   N|                Y|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R15W8DZ57HXZ28|          5|           19|         20|   N|                Y|
|R18UIVCYVG6HKQ|          3|           13|         20|   N|                N|
|R1Y6ZYTARUH13D|          5|           14|         20|   N|                N|
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R26OQY0D8FB0TN|          1|           16|         20|   N|                Y|
|R1N1YR7XNB54BF|          5|           20|         20|   N|     

In [48]:
#Deliverable 2.5
#For both vine_Y_df and vine_N_df, determine:
#1) Total number of reviews
vine_Y_Total = vine_Y_df.count()
vine_N_Total = vine_N_df.count()

#2) Number of 5-star reviews
vine_Y_5star_Total = vine_Y_df.filter(vine_Y_df.star_rating == '5').count()
vine_N_5star_Total = vine_N_df.filter(vine_N_df.star_rating == '5').count()

#3) Percent of 5-star reviews
vine_Y_5star_percent = ( vine_Y_5star_Total / vine_Y_Total ) * 100
vine_N_5star_percent = ( vine_N_5star_Total / vine_N_Total ) * 100

print('Vine Paid Subscribers')
print('Total 5-Star Reviews for Vine Paid Subscribers is ' + str(vine_Y_5star_Total))
print('Total Reviews for Vine Paid Subscribers is ' + str(vine_Y_Total))
print('Percentage of 5-Star Reviews is ' + str(round(vine_Y_5star_percent,2)) + '%')
print('----------------------------------------------------------')
print('Vine Un-Paid Non-Subscribers')
print('Total 5-Star Reviews is ' + str(vine_N_5star_Total))
print('Total Reviews is ' + str(vine_N_Total))
print('Percentage of 5-Star Reviews is ' + str(round(vine_N_5star_percent,2)) + '%')

Vine Paid Subscribers
Total 5-Star Reviews for Vine Paid Subscribers is 48
Total Reviews for Vine Paid Subscribers is 94
Percentage of 5-Star Reviews is 51.06%
----------------------------------------------------------
Vine Un-Paid Non-Subscribers
Total 5-Star Reviews is 15663
Total Reviews is 40471
Percentage of 5-Star Reviews is 38.7%
